<a href="https://colab.research.google.com/github/Edivaldo16/projeto_etl/blob/main/Transformacao_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformação de dados - Parte 1





In [1]:
import pandas as pd

In [ ]:
!pip install pandera

In [3]:
import pandera as pa

In [5]:
# Importando arquivos diretamente do computador
from google.colab import files
arq = files.upload()

Saving Ocorrencia_mod2.csv to Ocorrencia_mod2.csv


In [6]:
valores_ausentes = ['**','###!','####','****','*****','NULL']

In [7]:
df = pd.read_csv('Ocorrencia_mod2.csv', sep = ";", parse_dates=['ocorrencia_dia'], dayfirst = True, na_values = valores_ausentes, encoding='latin-1')
# na_values: Usado para substituir os valores das variáveis por NA

In [ ]:
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0


In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo":pa.Column(pa.Int,required=False),
        # Estou informando para a função ignorar essa coluna, já que ela não 
        # existe no arquivo. Serve também, caso eu queira excluir alguma coluna.
        # Por "default" required = True
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True), # Agora a coluna "ocorrencia_uf" poderá receber dados nulos
        # Informando o dado mínimo e máximo que vem na coluna
        "ocorrencia_aerodromo":pa.Column(pa.String, nullable=True), # Agora a coluna "ocorrencia_uf" poderá receber dados nulos
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable = True),
        # Uma forma para fazer tratamento na coluna 'ocorrencia_hora'
        "ocorrencia_hora":pa.Column(pa.String, nullable = True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [ ]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
6110,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,80257,ACIDENTE,DOM AQUINO,MT,NaN,2021-08-18,16:00:00,0
6113,80259,80259,INCIDENTE,CAMPINAS,SP,SBKP,2021-08-18,12:45:00,0


In [ ]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

## Usando comando 'iloc'

In [ ]:
# Observações:
# df.loc[label] Conceito label
# df.iloc[índice] Conceito índice (Aceita valores negativos e o "range" sempre é menos 1)

df.iloc[10:12]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,39789,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,NaN,2010-01-10,21:30:00,0
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8


## Localizando dados de uma coluna


In [ ]:
df.loc[:,'ocorrencia_uf'] 

0       RJ
1       PA
2       RJ
3       MT
4       RS
        ..
6110    RO
6111    PA
6112    MT
6113    SP
6114    SP
Name: ocorrencia_uf, Length: 6115, dtype: object

## Verificação dos valores nulos

In [ ]:
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        2456
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [ ]:
df.isnull().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        2456
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [ ]:
df.ocorrencia_uf.isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
6110    False
6111    False
6112    False
6113    False
6114    False
Name: ocorrencia_uf, Length: 6115, dtype: bool

In [ ]:
filtro = df.ocorrencia_uf.isnull() # Onde é que valor "nulo" na coluna "ocorrencia_uf"
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NaN,NaN,2013-09-02,02:54:00,0
5855,79956,79956,ACIDENTE,NÃO IDENTIFICADA,NaN,NaN,2021-02-26,11:00:00,0


In [ ]:
df.count()

codigo_ocorrencia           6115
codigo_ocorrencia2          6115
ocorrencia_classificacao    6115
ocorrencia_cidade           6115
ocorrencia_uf               6113
ocorrencia_aerodromo        3659
ocorrencia_dia              6115
ocorrencia_hora             6114
total_recomendacoes         6115
dtype: int64

## Filtros

In [ ]:
# Ocorrências com mais de 10 recomendações

filtro = df.total_recomendacoes > 10
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
63,39992,39992,ACIDENTE,SÃO PAULO,SP,SBMT,2010-02-17,20:07:00,11
326,42250,42250,ACIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-08-12,12:26:00,19
530,43471,43471,ACIDENTE,SÃO PAULO,SP,NaN,2010-12-14,20:15:00,11
727,44377,44377,ACIDENTE,ITÁPOLIS,SP,SDIO,2011-04-09,14:50:00,11
872,44796,44796,ACIDENTE,RECIFE,PE,SBRF,2011-07-13,09:54:00,23
884,44888,44888,INCIDENTE GRAVE,SÃO JOSÉ DOS PINHAIS,PR,SBCT,2011-07-15,17:05:00,11
1062,45554,45554,INCIDENTE GRAVE,UBERLÂNDIA,MG,SBUL,2011-11-15,02:15:00,20
1972,47938,47938,INCIDENTE,BRASÍLIA,DF,SBBR,2013-04-13,18:00:00,11
2797,52265,52265,ACIDENTE,SANTOS,SP,NaN,2014-08-13,13:03:00,13
3932,66432,66432,INCIDENTE GRAVE,VITÓRIA,ES,NaN,2017-02-21,11:47:00,12


In [ ]:
# Ocorrências com mais de 10 recomendações na coluna "ocorrencia_cidade"

filtro = df.total_recomendacoes > 10
# df.loc[filtro,'ocorrencia_cidade'] # Só aparece as informações sem ser no fortato tabela
df.loc[filtro,['ocorrencia_cidade','total_recomendacoes']] # Quando colocasse a coluna entre [], os valores 
# aprecem em formato de tabela

,ocorrencia_cidade,total_recomendacoes
63,SÃO PAULO,11
326,RIO DE JANEIRO,19
530,SÃO PAULO,11
727,ITÁPOLIS,11
872,RECIFE,23
884,SÃO JOSÉ DOS PINHAIS,11
1062,UBERLÂNDIA,20
1972,BRASÍLIA,11
2797,SANTOS,13
3932,VITÓRIA,12


In [ ]:
# Ocorrências cuja classificação == INCIDENTE GRAVE

filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro, ['ocorrencia_cidade','total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
7,CANUTAMA,3
8,CASCAVEL,2
9,PARÁ DE MINAS,0
29,CURITIBA,2
35,PALMAS,0
...,...,...
6090,MARAÚ,0
6095,ELDORADO DO SUL,0
6097,CAXIAS DO SUL,0
6098,CHAPADÃO DO SUL,0


In [ ]:
# Imprimindo todas as colunas baseadas no filtro acima:

df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
29,39709,39709,INCIDENTE GRAVE,CURITIBA,PR,SBBI,2010-01-23,16:36:00,2
35,39487,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
...,...,...,...,...,...,...,...,...,...
6090,80221,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6095,80228,80228,INCIDENTE GRAVE,ELDORADO DO SUL,RS,SIXE,2021-08-05,15:55:00,0
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0


In [ ]:
# Ocorrências cuja classificação == INCIDENTE GRAVE e o estado SP

filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
136,40270,40270,INCIDENTE GRAVE,PIRASSUNUNGA,SP,SDPY,2010-04-15,20:15:00,0
171,40620,40620,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2010-05-06,15:50:00,0
219,41411,41411,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-06-08,19:30:00,2
297,42289,42289,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-07-29,20:06:00,0
314,42323,42323,INCIDENTE GRAVE,SÃO JOSÉ DO RIO PRETO,SP,SBSR,2010-08-05,12:01:00,0
...,...,...,...,...,...,...,...,...,...
5949,80056,80056,INCIDENTE GRAVE,FERNANDÓPOLIS,SP,NaN,2021-04-28,11:20:00,0
6000,80114,80114,INCIDENTE GRAVE,LINS,SP,SWXQ,2021-06-05,19:43:00,0
6007,80124,80124,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2021-06-10,20:30:00,0
6081,80200,80200,INCIDENTE GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2021-07-25,12:25:00,0


In [ ]:
# Ocorrências cuja classificação == INCIDENTE GRAVE ou estado SP

filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 | filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
...,...,...,...,...,...,...,...,...,...
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0
6113,80259,80259,INCIDENTE,CAMPINAS,SP,SBKP,2021-08-18,12:45:00,0


In [ ]:
# Ocorrências cuja classificação == INCIDENTE GRAVE ou classificação == INCIDENTE e o estado SP - Forma 1
# Se digitar sem os parêntesis vai dar errado pq o programa precisa entender a prioridades das operações
filtro1 = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE')
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...,...
6085,80208,80208,INCIDENTE,SÃO PAULO,SP,SBMT,2021-07-27,19:45:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0
6113,80259,80259,INCIDENTE,CAMPINAS,SP,SBKP,2021-08-18,12:45:00,0


In [ ]:
# Ocorrências cuja classificação == INCIDENTE GRAVE ou classificação == INCIDENTE e o estado SP - Forma 2

filtro1 = df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE','INCIDENTE']) 
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...,...
6085,80208,80208,INCIDENTE,SÃO PAULO,SP,SBMT,2021-07-27,19:45:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0
6113,80259,80259,INCIDENTE,CAMPINAS,SP,SBKP,2021-08-18,12:45:00,0


In [ ]:
# Ocorrências cujas cidades começam com a letra "c"

filtro = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
20,40310,40310,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2010-01-18,19:32:00,0
...,...,...,...,...,...,...,...,...,...
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0
6100,80235,80235,INCIDENTE,CORUMBÁ,MS,SBCR,2021-08-09,20:55:00,0
6113,80259,80259,INCIDENTE,CAMPINAS,SP,SBKP,2021-08-18,12:45:00,0


In [ ]:
# Ocorrências cujas cidades terminam com a letra "a"

filtro = df.ocorrencia_cidade.str[-1] == 'A'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
12,40414,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
25,39772,39772,INCIDENTE,UBERLÂNDIA,MG,NaN,2010-01-20,18:58:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
...,...,...,...,...,...,...,...,...,...
6093,80227,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0
6094,80225,80225,INCIDENTE,CURITIBA,PR,SBBI,2021-08-04,13:45:00,0
6102,80240,80240,INCIDENTE,GOIÂNIA,GO,SBNV,2021-08-10,14:15:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0


In [ ]:
# Ocorrências cujas cidades terminam com os caracteres "MA"

filtro = df.ocorrencia_cidade.str[-2:] == 'MA' # Do penúltimo em diante "MA"
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
408,43062,43062,ACIDENTE,NOVO GAMA,GO,NaN,2010-09-25,16:30:00,1
688,44198,44198,ACIDENTE,BURITAMA,SP,NaN,2011-03-19,11:18:00,0
1035,51785,51785,INCIDENTE,BURITAMA,SP,SIBX,2011-10-25,19:15:00,0
1377,45939,45939,ACIDENTE,UMUARAMA,PR,NaN,2012-05-30,19:00:00,0
1401,46180,46180,ACIDENTE,AURIFLAMA,SP,NaN,2012-06-16,19:30:00,0
2099,49113,49113,INCIDENTE GRAVE,NOVA LIMA,MG,NaN,2013-06-15,12:00:00,4
2102,48799,48799,ACIDENTE,CANUTAMA,AM,NaN,2013-06-18,21:30:00,0
2646,80059,80059,INCIDENTE,IVINHEMA,MS,NaN,2014-04-27,11:30:00,0


In [ ]:
# Ocorrências cujas cidades contem com os caracteres "MA"

filtro = df.ocorrencia_cidade.str.contains('MA') # Onde tiver "MA", independente da posição, imprima
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
35,39487,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
78,39916,39916,INCIDENTE GRAVE,MANAUS,AM,SBEG,2010-03-07,19:35:00,2
...,...,...,...,...,...,...,...,...,...
6075,80205,80205,INCIDENTE,MANAUS,AM,SBEG,2021-07-23,11:15:00,0
6086,80212,80212,INCIDENTE,AMAPÁ,AP,NaN,2021-07-28,18:00:00,0
6090,80221,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6093,80227,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0


In [ ]:
# Ocorrências cujas cidades contem com os caracteres "MA" ou "AL"

filtro = df.ocorrencia_cidade.str.contains('MA|AL') # Onde tiver "MA", independente da posição, imprima
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
17,39810,39810,INCIDENTE,SALVADOR,BA,NaN,2010-01-15,18:23:00,0
...,...,...,...,...,...,...,...,...,...
6075,80205,80205,INCIDENTE,MANAUS,AM,SBEG,2021-07-23,11:15:00,0
6086,80212,80212,INCIDENTE,AMAPÁ,AP,NaN,2021-07-28,18:00:00,0
6090,80221,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6093,80227,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0


In [ ]:
# Ocorrências do ano de 2015

filtro = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2998,52992,52992,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2015-01-01,11:40:00,3
2999,52979,52979,ACIDENTE,IVINHEMA,MS,NaN,2015-01-02,14:50:00,0
3000,53073,53073,INCIDENTE,TEFÉ,AM,SBTF,2015-01-02,16:28:00,0
3001,53074,53074,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-01-02,20:34:00,0
3002,52976,52976,ACIDENTE,TOLEDO,PR,SBTD,2015-01-04,22:04:00,6
...,...,...,...,...,...,...,...,...,...
3464,60632,60632,INCIDENTE GRAVE,ITABERÁ,SP,NaN,2015-12-24,14:00:00,0
3465,60600,60600,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-25,19:00:00,0
3466,60642,60642,INCIDENTE,SÃO FRANCISCO DO SUL,SC,SSSS,2015-12-26,16:00:00,0
3467,60631,60631,ACIDENTE,MAÇAMBARÁ,RS,NaN,2015-12-28,19:00:00,2


In [ ]:
# Ocorrências do ano de 2015 e mês 12 - Forma 1

filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3430,53573,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
3431,60601,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
3432,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3433,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [ ]:
# Ocorrências do ano de 2015 e mês 12 - Forma 2

filtro = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 12)
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3430,53573,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
3431,60601,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
3432,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3433,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [ ]:
# Ocorrências do ano de 2015, mês 12 e dia 8

filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
filtro3 = df.ocorrencia_dia.dt.day == 8

df.loc[filtro1 & filtro2 & filtro3]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3440,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3441,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [ ]:
# Ocorrências do ano de 2015, mês 12 e dos dias 3 a 8

filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
filtro3 = df.ocorrencia_dia.dt.day >= 3  
filtro4 = df.ocorrencia_dia.dt.day <= 8
df.loc[filtro1 & filtro2 & filtro3 & filtro4]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3440,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3441,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [ ]:
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)
# Primeiro concatenei o dia e a hora sendo que tive que transformar dia em str para poder concatenar
# Daí, para trabalhar com a coluna "ocorrencia_dia_hora" como uma "data" usei a expressão "pd.to_datetime()"

In [ ]:
filtro1 = df.ocorrencia_dia_hora >= '2015-12-03 11:00:00'
filtro2 = df.ocorrencia_dia_hora <= '2015-12-08 13:00:00'
df[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3,2015-12-06 13:10:00
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
3441,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


## Agrupamento de Dados

In [ ]:
# Ocorrências do ano de 2015 e mês 03

filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 3

df201503 = df.loc[filtro1 & filtro2]
df201503

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3088,53183,53183,ACIDENTE,AMAPORÃ,PR,NaN,2015-03-02,23:00:00,0,2015-03-02 23:00:00
3089,53120,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
3090,53109,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
3091,53112,53112,ACIDENTE,MOGI GUAÇU,SP,NaN,2015-03-06,21:00:00,0,2015-03-06 21:00:00
3092,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
3093,53167,53167,INCIDENTE,MARABÁ,PA,SBMA,2015-03-10,17:33:00,0,2015-03-10 17:33:00
3094,53596,53596,INCIDENTE,ITAPEMA,SC,NaN,2015-03-10,21:35:00,0,2015-03-10 21:35:00
3095,53149,53149,ACIDENTE,TABATINGA,AM,SBTT,2015-03-11,23:25:00,0,2015-03-11 23:25:00
3096,53148,53148,ACIDENTE,ARARAS,SP,SDEH,2015-03-12,12:40:00,0,2015-03-12 12:40:00
3097,53153,53153,INCIDENTE,PORTO ALEGRE,RS,SBPA,2015-03-13,15:45:00,0,2015-03-13 15:45:00


In [ ]:
df201503.count()

codigo_ocorrencia           37
codigo_ocorrencia2          37
ocorrencia_classificacao    37
ocorrencia_cidade           37
ocorrencia_uf               37
ocorrencia_aerodromo        21
ocorrencia_dia              37
ocorrencia_hora             37
total_recomendacoes         37
ocorrencia_dia_hora         37
dtype: int64

In [ ]:
df201503.groupby(['codigo_ocorrencia']).count() # Agrupando pelo "codigo_ocorrencia"

In [ ]:
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count() 

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

In [ ]:
df201503.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
dtype: int64

In [ ]:
# Ordem Crescente

df201503.groupby(['ocorrencia_classificacao']).size().sort_values()

ocorrencia_classificacao
INCIDENTE GRAVE     5
ACIDENTE           15
INCIDENTE          17
dtype: int64

In [ ]:
# Ordem Decrescente

df201503.groupby(['ocorrencia_classificacao']).size().sort_values(ascending = False)

ocorrencia_classificacao
INCIDENTE          17
ACIDENTE           15
INCIDENTE GRAVE     5
dtype: int64

In [8]:
# Eventos no ano de 2010 na região Sudeste

filtro1 = df.ocorrencia_dia.dt.year == 2010
filtro2 = df.ocorrencia_uf.isin(['SP','MG','ES','RJ'])
dfsudeste2010 = df.loc[filtro1 & filtro2]
dfsudeste2010

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
...,...,...,...,...,...,...,...,...,...
541,43722,43722,ACIDENTE,CAJOBI,SP,NaN,2010-12-23,11:00:00,0
544,43548,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0
548,43724,43724,ACIDENTE,COSMÓPOLIS,SP,NaN,2010-12-29,12:00:00,0
550,43730,43730,INCIDENTE,VITÓRIA,ES,NaN,2010-12-30,18:10:00,0


In [9]:
dfsudeste2010.groupby(['ocorrencia_classificacao']).size() # Quantas vezes cada "ocorrencia_classificacao" ocorre???

ocorrencia_classificacao
ACIDENTE            40
INCIDENTE          170
INCIDENTE GRAVE     18
dtype: int64

In [10]:
dfsudeste2010.count() # codigo_ocorrencia: 40 + 170 + 18 = 218

codigo_ocorrencia           228
codigo_ocorrencia2          228
ocorrencia_classificacao    228
ocorrencia_cidade           228
ocorrencia_uf               228
ocorrencia_aerodromo        127
ocorrencia_dia              228
ocorrencia_hora             228
total_recomendacoes         228
dtype: int64

In [11]:
dfsudeste2010.groupby(['ocorrencia_classificacao','ocorrencia_uf']).size() # Quantas vezes cada "ocorrencia_classificacao" 
# ocorre em cada "ocorrencia_uf"???

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  MG                7
                          RJ                9
                          SP               24
INCIDENTE                 ES                5
                          MG               39
                          RJ               52
                          SP               74
INCIDENTE GRAVE           MG                5
                          RJ                4
                          SP                9
dtype: int64

In [13]:
dfsudeste2010.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False) # Agrupe o número de acidentes, por cidade,
# em ordem decrescente

ocorrencia_cidade
RIO DE JANEIRO           47
SÃO PAULO                33
GUARULHOS                18
BELO HORIZONTE           17
CAMPINAS                 13
MACAÉ                    12
CONFINS                   7
UBERLÂNDIA                5
VITÓRIA                   4
JUIZ DE FORA              4
SÃO JOSÉ DO RIO PRETO     3
SÃO JOSÉ DOS CAMPOS       3
MARÍLIA                   3
ARARAQUARA                2
MONTES CLAROS             2
CAMPOS DOS GOYTACAZES     2
DIVINÓPOLIS               2
IPATINGA                  2
ITANHAÉM                  2
JUNDIAÍ                   2
POÇOS DE CALDAS           2
RIBEIRÃO PRETO            2
SOROCABA                  2
UBERABA                   2
BAURU                     2
AREALVA                   2
BOITUVA                   1
BOCAINA                   1
BOM JESUS DO AMPARO       1
BOM JESUS DO GALHO        1
BEBEDOURO                 1
BRAGANÇA PAULISTA         1
BROTAS                    1
CAJOBI                    1
ARRAIAL DO CABO           1
CA

In [15]:
filtro = dfsudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2010.loc[filtro].total_recomendacoes.sum() # Somar o "total_recomendacoes" que o RIO DE JANEIRO teve

25

In [16]:
filtro1= dfsudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
filtro2 = dfsudeste2010.total_recomendacoes > 0
dfsudeste2010.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
119,40190,40190,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2010-04-06,02:35:00,3
274,41750,41750,ACIDENTE,RIO DE JANEIRO,RJ,SBJR,2010-07-12,19:53:00,3
326,42250,42250,ACIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-08-12,12:26:00,19


In [17]:
dfsudeste2010.groupby(['ocorrencia_cidade']).total_recomendacoes.sum() # Realize a soma do total de recomendações por cidade

ocorrencia_cidade
AMERICANA                 0
ARARAQUARA                0
ARAÇATUBA                 0
AREALVA                   0
ARRAIAL DO CABO           0
BAURU                     0
BEBEDOURO                 0
BELO HORIZONTE            2
BOCAINA                   0
BOITUVA                   1
BOM JESUS DO AMPARO       0
BOM JESUS DO GALHO        1
BRAGANÇA PAULISTA         9
BROTAS                    0
CAJOBI                    0
CAMANDUCAIA               9
CAMPINAS                  0
CAMPOS DOS GOYTACAZES     0
CASIMIRO DE ABREU         0
CONFINS                   0
COROMANDEL                0
COSMÓPOLIS                0
DIAMANTINA                0
DIVINÓPOLIS               0
GUARULHOS                 0
IPATINGA                  0
IPERÓ                     3
ITANHAÉM                  1
ITÁPOLIS                  0
JUIZ DE FORA              0
JUNDIAÍ                   7
LINHARES                  0
MACAÉ                     0
MANGARATIBA               0
MARÍLIA                   0
MO

In [18]:
dfsudeste2010.groupby(['ocorrencia_aerodromo'], dropna=False).total_recomendacoes.sum() # O dropna falso faz retornar elementos
# que não são informados

ocorrencia_aerodromo
SBAE     0
SBAQ     0
SBBH     0
SBBP     9
SBBU     0
SBCF     0
SBGL     3
SBGR     0
SBIP     0
SBJD     7
SBJR     3
SBKP     0
SBME     0
SBMK     0
SBML     0
SBMT    27
SBPC     0
SBPR     1
SBRJ    19
SBRP     0
SBSJ     0
SBSP     0
SBSR     0
SBUL     0
SBUR     0
SBVT     0
SDAM     0
SDOI     1
SDPW     0
SDPY     0
SNDT     0
SNDV     0
SNXT     0
NaN     45
Name: total_recomendacoes, dtype: int64

In [19]:
dfsudeste2010.groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

ocorrencia_cidade
AMERICANA                 0
ARARAQUARA                0
ARAÇATUBA                 0
AREALVA                   0
ARRAIAL DO CABO           0
BAURU                     0
BEBEDOURO                 0
BELO HORIZONTE            2
BOCAINA                   0
BOITUVA                   1
BOM JESUS DO AMPARO       0
BOM JESUS DO GALHO        1
BRAGANÇA PAULISTA         9
BROTAS                    0
CAJOBI                    0
CAMANDUCAIA               9
CAMPINAS                  0
CAMPOS DOS GOYTACAZES     0
CASIMIRO DE ABREU         0
CONFINS                   0
COROMANDEL                0
COSMÓPOLIS                0
DIAMANTINA                0
DIVINÓPOLIS               0
GUARULHOS                 0
IPATINGA                  0
IPERÓ                     3
ITANHAÉM                  1
ITÁPOLIS                  0
JUIZ DE FORA              0
JUNDIAÍ                   7
LINHARES                  0
MACAÉ                     0
MANGARATIBA               0
MARÍLIA                   0
MO

In [20]:
filtro = dfsudeste2010.total_recomendacoes > 0
dfsudeste2010.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

ocorrencia_cidade
BELO HORIZONTE            2
BOITUVA                   1
BOM JESUS DO GALHO        1
BRAGANÇA PAULISTA         9
CAMANDUCAIA               9
IPERÓ                     3
ITANHAÉM                  1
JUNDIAÍ                   7
NOVO HORIZONTE            3
RESENDE                   8
RIO DE JANEIRO           25
SÃO BERNARDO DO CAMPO     2
SÃO PAULO                41
TIMÓTEO                   3
Name: total_recomendacoes, dtype: int64

In [26]:
# Total de recomendações agrupando por cidade e por mês

dfsudeste2010.loc[filtro].groupby(['ocorrencia_cidade',dfsudeste2010.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

ocorrencia_cidade      ocorrencia_dia
BELO HORIZONTE         2                  1
                       3                  1
BOITUVA                7                  1
BOM JESUS DO GALHO     12                 1
BRAGANÇA PAULISTA      11                 9
CAMANDUCAIA            7                  9
IPERÓ                  1                  3
ITANHAÉM               9                  1
JUNDIAÍ                5                  5
                       6                  2
NOVO HORIZONTE         7                  3
RESENDE                4                  8
RIO DE JANEIRO         4                  3
                       7                  3
                       8                 19
SÃO BERNARDO DO CAMPO  1                  2
SÃO PAULO              1                  8
                       2                 14
                       6                  2
                       11                 6
                       12                11
TIMÓTEO                2              